In [2]:
from driftdb.connectors import GithubConnector
from driftdb.version import version
print(version)
from dotenv import load_dotenv
from github import Github
import os

load_dotenv("../../.env")

gh_token = os.getenv("GH_TOKEN")
if gh_token is None:
    print("GitHub token not found! Create a .env file a the root with a GH_TOKEN variable.")
    exit(1)
github_client = Github(gh_token, timeout=60)
repo_name = os.getenv("REPO") or "gh_org/repo"
github_connector = GithubConnector(github_client=github_client, github_repository_name=repo_name, assignees=["Samox"])


0.0.5-a3


In [3]:

import pandas as pd


def formatDF(dict):
    df = pd.DataFrame(dict)
    df['unique_key'] = df.apply(lambda row: row['date'] + '-' + row['name'], axis=1)
    column_order = ['unique_key'] + [col for col in df.columns if col != 'unique_key']
    df = df.reindex(columns=column_order)
    return df


table_name = "path/to/metric_name_15"


# Store metric for the first time
dataMonth1 = {"name": ["Alice", "Bob", "Charlie"], "date": ["2022-12","2023-01","2023-01"], "age": [25, 30, 35]}
github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth1),
    table_name= table_name,
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: DRIFT
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/9dedc6b3bde719fec29abeb538b5ef6c0a652b84


In [4]:
# ## Introduce new data for 2023-02
dataMonth2 = {"name": ["Alice", "Bob", "Charlie", "Didier", "Philipe", "Antoine"], "date": ["2022-12","2023-01","2023-01","2023-02","2023-02","2023-02"], "age": [25, 30, 35, 40, 40, 40]}

github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth2),
    table_name= table_name,
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: NEW DATA
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/9ef8ff500188cf80421c04a8cbddaa5107dcd45b


In [5]:
# ## Introduce new data for 2023-03 and a drift on 2020-02
dataMonth3 = {"name": ["Alice", "Bob", "Charlie", "Didier", "Philipe", "Antoine", "Clement", "Cyril", "Victor"], "date": ["2022-12","2023-01","2023-01","2023-02","2023-02","2023-02","2023-03","2023-03","2023-03"], "age": [25, 30, 36, 40, 42, 40, 45, 45, 46]}

github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth3),
    table_name= table_name,
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: NEW DATA
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/d1475721c2ac980b9db8e2b05069cabeada98a00
driftdb.connectors.github_connector - INFO - Update: DRIFT
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/6cb35036f07dd921d86e137ed6851a5432a17bd6


In [6]:
# ## No new data. Adds a drift for Philipe, and remove all other drifts
from driftdb.drift_evaluator.drift_evaluators import AlertDriftEvaluator


dataMonth3And1Day = {"name": ["Alice", "Bob", "Charlie", "Didier", "Philipe", "Antoine", "Clement", "Cyril", "Victor"], "date": ["2022-12","2023-01","2023-01","2023-02","2023-02","2023-02","2023-03","2023-03","2023-03"], "age": [25, 30, 35, 40, 42, 40, 45, 45, 46]}
github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth3And1Day),
    table_name= table_name,
    drift_evaluator=AlertDriftEvaluator()
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: DRIFT
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/03b345b8f209b39d5016bf0e59fd2fb42c0970cf
driftdb.connectors.github_connector - INFO - Issue created: https://github.com/Samox/data-history/issues/241


In [7]:
# ## No new data. Adds a drift for Philipe, and remove all other drifts
dataMonth3And2Day = {"name": ["Alice", "Alixe", "Bob", "Charlie", "Didier", "Philipe", "Antoine", "Clement", "Cyril", "Victor"], "date": ["2022-12","2022-12","2023-01","2023-01","2023-02","2023-02","2023-02","2023-03","2023-03","2023-03"], "age": [25, 25, 30, 35, 40, 42, 40, 45, 45, 46]}

github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth3And2Day),
    table_name= table_name,
    drift_evaluator=AlertDriftEvaluator()
)


driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: DRIFT
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/a38b0fe1aa646d6907b6154bd91cad0b4c050e26
driftdb.connectors.github_connector - INFO - Issue created: https://github.com/Samox/data-history/issues/242
